# TensorFlow example

[TensorFlow](https://www.tensorflow.org/) is one of the many deep learning packages. It is written in C++ but the API is in Python. Many of the functions mirror NumPy.

It should be noted that-- as with most of these "deep learning" toolkits-- the library can be used to do all sorts of machine learning. It is *not* limited to simply neural networks. 

TensorFlow/Theano/Caffe/Torch/etc are really just symbolic math libraries. The advantage of using these libraries is that they are designed to be run on machines with GPUs. So--like Spark-- they allow us to run algorithms faster by distributing the calculations over several processors (in this case GPUs and multi-core CPUs). Additionally, they symbolically calculate derivatives of functions which gives a better and faster implementation of back-propagation.

We'll be using [Keras](https://keras.io/) as the actual interface to Tensorflow/Theano/etc. Keras provides an abstraction layer above TensorFlow/Theano. It will handle a lot of the heavy lifting so that we can just define the neural network parameters and it can handle the actual TensorFlow coding.

I should also note that TensorFlow has officially supported [SKFlow](https://github.com/tensorflow/tensorflow/tree/master/tensorflow/contrib/learn/python/learn) which is a Sk-Learn binding to TensorFlow. This might be something that takes up traction as it gives you the pre-defined routines of sk-learn with the GPU scalability of TensorFlow. Examples of the typical sk-learn datasets are [here](https://github.com/tensorflow/tensorflow/tree/master/tensorflow/examples/learn).

In [1]:
# Example of TensorFlow library
import tensorflow as tf

## Tensors

In the case of all of these packages, a tensor is just an abstraction for a N-dimensional matrix (N rank). So a scalar is a Tensor of dimension 0. A vector is a tensor of dimension 1, a matrix is a tensor of dimension 2. Tensors become useful for things like image processing because there are 3 color channels (red, green, blue). So a color image is $n \times m \times 3$ matrix or a 3-Tensor (n pixels width, m pixels height). A video would be a 4-Tensor $n \times m \times 3 \times t$ where t is the index of the video frame. So tensors help us to store our data in a logical format and process them in a logical way (e.g. we know what a dimension refers to in the real world).

Here's from the TensorFlow documentation:

 Tensor Rank |	Mathematical Construct |	Python example 
 :----: | -------     | ------
 0    |	Scalar (magnitude only) |	s = 123 
 1 	  | Vector (magnitude and direction) | 	v = [1.1, 2.2, 3.3] 
 2 	  | Matrix (table of numbers) 	| m = [[1, 2, 3], [4, 5, 6], [7, 8, 9]] 
 3 	  | 3-Tensor (cube) |	t = [[[2], [4], [6]], [[8], [10], [12]], [[14], [16], [18]]] 

In [2]:
# declare two symbolic floating-point scalars
a = tf.placeholder(tf.float32)
b = tf.placeholder(tf.float32)

## Note that "a" is just a place holder (i.e. a variable) for a float32 scalar.

We'll feed data into "a" later.

In [3]:
print(a)    # "Placeholder:#" is the internal name for the node of type float32

Tensor("Placeholder:0", dtype=float32)


In [4]:
# create a simple symbolic expression using the add and multiply function
fun = tf.add(a, b) + tf.multiply(a,b)  # I could also do fun = a+b + a*b

## TensorFlow sessions

TensorFlow doesn't actually execute anything until a session is created and run.  So it's like Spark's lazy execution. Setting up a session instructs TensorFlow to make the plan of how it will use the resources (GPU/CPU). TensorFlow automatically determines the best way to assign a GPU or CPU device to an operation depending on what it has available. That's called the *graph*. Then, session.run() will actually cause the graph to execute and return a result.

So in the graph our nodes are operations, variables, and constants. Our edges are tensors-- n-dimensional data arrays. Tensors _flow_ between nodes. Hence, tensor flow.

Here's what our graph looks like. Each worker (GPU/CPU) will get a copy of the graph and feed data into it.

![Graph](../tfGraph.png)

In [5]:
sess = tf.Session()   # A new session is created. Still waiting for graph to be defined.

## Write the graph

We don't have to write out the graph, but we're doing this to demonstrate the use of [TensorBoard](https://www.tensorflow.org/get_started/summaries_and_tensorboard). TensorBoard is a built in GUI that allows us to monitor our TensorFlow graphs. For this example, we just write the proposed graph to file.

To view the file, go to the command prompt and type:

> tensorboard --logdir="./graphs" --port 6006<br><br>
Then have your browser go to http://localhost:6006 and click on the "Graphs" tab.

It should look something like this:

![image](tfGraph2.png)

In [6]:
writer = tf.summary.FileWriter('./graphs', sess.graph) # Writes our graph to a file for TensorBoard

In [7]:
# Let's define the data we want to pass to this graph
# bind 5 to 'a', 3 to 'b'
binding = {a: 5, b: 3}

In [8]:
result = sess.run(fun, feed_dict=binding)  # Run the graph 'fun' using the data dictionary 'binding'

# Results comes back in result
print(result)
print(type(result))

23.0
<type 'numpy.float32'>


In [9]:
writer.close()
sess.close() # Close the current session

## Interactive sessions

[Interactive sessions](http://learningtensorflow.com/lesson5/) make it easier for us to try things out without having to keep referring to the session variable.

In [10]:
sess = tf.InteractiveSession()

In [11]:
x = tf.constant([1, -2, 3.2], tf.float32)
y = tf.constant([-1, 2, -3], tf.float32)  # TF duck-types so this will be int vector if not explicitly defined as float32

## Dot product of two vectors

In [12]:
dot_product = tf.reduce_sum(tf.multiply(x, y))

In [14]:
d = dot_product.eval()

print(d)
print(type(d))

-14.6
<type 'numpy.float32'>


In [15]:
sess.close() # Close the current session

## Simple sum - Lazy Evaluation of the Graph

In [16]:
sess = tf.InteractiveSession()

In [17]:
add = tf.add(-4,6)

## Notice that "add" doesn't actually get evaluated at this step

Currently, add just holds a graph which anticipates loading int32. Even the shape of the output is undefined until execution time (it will be based on the actual data inputed).

In [18]:
print(add)

Tensor("Add_1:0", shape=(), dtype=int32)


In [19]:
add.eval()  # Now we execute the graph

2

In [20]:
sess.close()

## Using Python, Numpy, and TensorFlow to define our tensors

In [21]:
import numpy as np

# Define a 2x2 matrix in 3 different ways
m1 = [[1.0, 2.0], 
      [3.0, 4.0]]

m2 = np.array([[1.0, 2.0], 
               [3.0, 4.0]], dtype=np.float32)


m3 = tf.constant([[-1.0, 3.0], 
                  [3.3, -4.4]], tf.float32)

In [22]:
# Print the type for each matrix
print(type(m1))
print(type(m2))
print(type(m3))

<type 'list'>
<type 'numpy.ndarray'>
<class 'tensorflow.python.framework.ops.Tensor'>


In [25]:
# Create tensor objects out of the different types
t1 = tf.convert_to_tensor(m1, dtype=tf.float32)
t2 = tf.convert_to_tensor(m2, dtype=tf.float32)
t3 = m3 #tf.convert_to_tensor(m3, dtype=tf.float32)

In [26]:
# Notice that the types will be the same now
print(type(t1))
print(type(t2))
print(type(t3))

<class 'tensorflow.python.framework.ops.Tensor'>
<class 'tensorflow.python.framework.ops.Tensor'>
<class 'tensorflow.python.framework.ops.Tensor'>


## Let's run more than one graph

If we use the "with" clause, then we don't need to specifically close the session. 

In [27]:
tfADD = tf.add(t1, t2)
tfADD2 = tf.subtract(t2, t3) + tf.exp(t1) # Subtract t2 and t2, then add exponential of t1

In [28]:
# Start a session to be able to run operations
with tf.Session() as sess:
    # Tell the session to evaluate the operation
    result = sess.run(tfADD)
    result2 = sess.run(tfADD2)

In [31]:
print(result)
print(type(result))

[[ 2.  4.]
 [ 6.  8.]]
<type 'numpy.ndarray'>


In [30]:
print(result2)

[[  4.71828175   6.38905621]
 [ 19.78553772  62.99815369]]


## Defining ranges

All of the standard numpy equivalents are here:

+ tf.ones(shape, dtype, name)
+ tf.zeros(shape, dtype, name)
+ tf.fill(dims, value_name)
+ tf.linspace(start, stop, num, name=None)
+ tf.range(start, limit=None, delta=1, dtype=None, name='range')

In [33]:
sess = tf.InteractiveSession()

In [34]:
print (tf.ones([2,3]).eval())

[[ 1.  1.  1.]
 [ 1.  1.  1.]]


In [35]:
print(tf.zeros([3,5]).eval())

[[ 0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.]]


In [36]:
print(tf.fill([2,2], 4.5).eval())   # 2 x 2 matrix filled with 4.5

[[ 4.5  4.5]
 [ 4.5  4.5]]


In [38]:
print(tf.linspace(2.0, 11.0, 4).eval())   # From 2 to 11 with exactly 4 elements

[  2.   5.   8.  11.]


In [43]:
print(tf.range(5).eval())   # From zero to 5 in steps of 1

[0 1 2 3 4]


In [57]:
print(tf.range(5,23,4).eval()) # From 5 to 23 in steps of 4

[ 5  9 13 17 21]


## Random numbers

Let's generate random number arrays

In [46]:
print(tf.random_normal([2, 3], mean=-1, stddev=4).eval())  # Normal distribution 2x3 matrix, mean -1, std = 4

[[ 3.03492737  0.04655898 -6.37584066]
 [-0.21875036 -2.02287245  0.89129341]]


In [39]:
sess.close()